In [2]:
!pip install boto3 langchain langchain_community -Uq


In [1]:

import boto3
import random
import dotenv
#load env from .env
dotenv.load_dotenv()
bedrock_runtime = boto3.client('bedrock-runtime')

In [2]:

from langchain_community.chat_models import BedrockChat
from langchain_core.messages import HumanMessage,AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder,HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [3]:

llm_sonnet = BedrockChat(model_id="anthropic.claude-3-sonnet-20240229-v1:0",
                  model_kwargs={"temperature": 0.2,
                                "top_k":250,
                                "max_tokens": 1024,
                                "top_p":0.95,
                                # "stop_sequences":['</response>']
                               },
                  streaming=False,callbacks=[StreamingStdOutCallbackHandler()])

llm_haiku = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0",
                  model_kwargs={"temperature": 0.2,
                                "top_k":250,
                                "max_tokens": 1024,
                                "top_p":0.5,
                                # "stop_sequences":['</response>']
                               },
                  streaming=False,callbacks=[StreamingStdOutCallbackHandler()])

llm_gpt= ChatOpenAI( model='gpt-3.5-turbo',
                     temperature= 0.2,
                    )

llm_gpt4= ChatOpenAI( model='gpt-4-0125-preview',
                     temperature= 0.2,
                    )

/Users/chuanxie/opt/anaconda3/envs/pyautogen/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


In [4]:
prompt = \
"""You are an very experienced software developer.
You will be provided with code snippet the following format:
<fim_prefix>starting code or comment</fim_prefix>
<fim_middle>code or commentto be completed </fim_middle>
<fim_suffix>ending code or comment</fim_suffix>

You need to complete the code or  explanation comment in the missing section between <fim_prefix> and <fim_suffix>. 
For examples:
<example_1>
Given the input: 
<fim_prefix>
# test.py
import random
def generate_id():
  
</fim_prefix>
<fim_suffix>
  return id
</fim_suffix>

The missing part is:
<fim_middle>
   id = "id-"+str(random.randint(1000,9999))
</fim_middle>
</example_1>

<example_2>
Given the input: 
<fim_prefix>
# test.cs
//create a funtion to generate random Id
public class IdGenerator
{{
    public static string GenerateId()
    {{
        var random = new Random();
</fim_prefix>      
<fim_suffix>
        return id.ToString();
    }}
}}
</fim_suffix>

The missing part is:
<fim_middle>
    var id = random.Next(100000, 999999);
</fim_middle>
</example_2>

<example_3>
Given the input: 
<fim_prefix>
# test.js
//创建一个函数随机生成一个id字符串
function generateUniqueId(){{
    const timestamp = Date.now();
  const randomNumber = Math.random();
  //将随机数
</<fim_prefix>
<fim_suffix>
     const hexadecimalString = randomNumber.toString(16);
  return `id-${{timestamp}}-${{hexadecimalString}}`;
}}
</fim_suffix>

The missing part is:
<fim_middle>
转成十六进制字符串
</fim_middle>
</example_3>

<example_4>
<fim_prefix>
# test.go
func max(num1, num2 int) int {{
   /* 声明局部变量 */
   var result int
   if (num1 > num2) {{
      result = num1
      }} else {{
</fim_prefix>
<fim_suffix>
   return result
}}
</fim_suffix>
The missing part is:
<fim_middle>
    result = num2
   }}
</fim_middle>
</example_4>


Given the input of code snippet writing in {code_lang}: 
<fim_prefix>{prefix}</fim_prefix>
<fim_middle>to be completed</fim_middle>
<fim_suffix>{suffix}</fim_suffix>

Your task is to consider the full snippet of code in <fim_prefix> and <fim_suffix>, and complete the missing part between them.
the missing part is the part between <fim_middle> and </fim_middle>. and it either be code snippet or explanation comment.
You must think hard to decide whether it is code snippet or explanation comment.
if it is code snippet, you must complete the code snippet.
if it is explanation comment, you must complete the explanation comment as the same language as the comment in prefix section
Skip the preamble, go straight into the answer.
Maker sure there is no syntax error.
The missing part is:
<fim_middle>
"""

In [5]:
prompt_template = ChatPromptTemplate.from_template(prompt)

In [6]:
prefix = """// AssetBundleGeneratorV2.cs
using System;
using System.Collections;
using System.Collections.Generic;
using UnityEditor;
using UnityEngine;
using System.IO;
using System.Linq;
using Newtonsoft.Json;
using UnityEditor.Build.Content;
using UnityEditor.Build.Pipeline;
using UnityEditor.Build.Pipeline.Interfaces;
using UnityEngine.Build.Pipeline;
using UnityEditor.Build.Pipeline.Tasks;
// using UnityEditor.AddressableAssets.Build.BuildPipelineTasks;
using UnityEngine.Networking;
using BuildCompression = UnityEngine.BuildCompression;

namespace NexgenDragon
{
	public partial class AssetBundleGeneratorV2 : AssetBundleGenerator
	{
		public static string OldBundlePath = "ArtData/OldBundles";
		public static int gHashLen = "d2e9405be3d60bbd575c97e8f33d".Length;

		public static bool miniMode = false;
		//	public static readonly string ASSETS_PATH_CONFIG_FILE = "assets_path_{0}.json";
		
		class CustomBuildParameters : BundleBuildParameters
		{
			public Dictionary<string, BuildCompression> PerBundleCompression { get; set; }

			public CustomBuildParameters(BuildTarget target, BuildTargetGroup group, string outputFolder) : base(target, group, outputFolder)
			{
				PerBundleCompression
"""

suffix = """
		// Override the GetCompressionForIdentifier method with new logic
			public override BuildCompression GetCompressionForIdentifier(string identifier)
			{
				BuildCompression value;
				if (PerBundleCompression.TryGetValue(identifier, out value))
					return value;
				return BundleCompression;
			}
		}

		private static void CompressBundles(string bundleType)
		{
			var bundlePath = string.Format(AssetBundleOutputPath, bundleType);
			
			var files = Directory.GetFiles(bundlePath);
			foreach (var bundle in files)
			{
				if (AssetConfig.BundleNeedDecompress(bundle) && !bundle.EndsWith(".meta"))
				{
					var tmpName = bundle + "_temp";
					File.Move(bundle,tmpName);
					var ret = LzmaLib.encodeLzmaFile(tmpName, bundle, 9, 20);
					File.Delete(tmpName);
					if (ret != 0)
					{
						NLogger.Error("bundle {0} compress error happened",bundle);
						return;
					}
				}
			}
		}

		public static void DeleteDirectory(string target_dir) {
            string[] files = Directory.GetFiles(target_dir);
            string[] dirs = Directory.GetDirectories(target_dir);
            foreach (string file in files) {
                NLogger.Log("AssetBundleGeneratorV2.DeleteDirectory filename = {0}", file);
                File.SetAttributes(@file, FileAttributes.Normal);
                File.Delete(file);
            }
            foreach (string dir in dirs) {
                DeleteDirectory(dir);
            }
            Directory.Delete(target_dir, false);
        }
		
		[MenuItem("NexgenDragon/AssetBundleToolV2/GenerateAssetBundleForAndroid",false,800)]
		public static void GenerateAssetBundleForAndroidTool()
		{
			GenerateAssetBundleForAndroid();
		}

        [MenuItem("NexgenDragon/AssetBundleToolV2/TestGenerateAssetBundleForAndroid",false,801)]
        public static void GenerateAssetBundleForAndroidTool2()
        {
	        MoBuildReport.Init("1.0.0");
			var target = BuildTarget.Android;
            AssetBundleOutputPath = ANDROID_BUNDLE_PATH;
            var buildGroup = BuildPipeline.GetBuildTargetGroup(target);
            // Construct the new parameters class
            var buildParams = new CustomBuildParameters(target, buildGroup, GetAssetBundleFolderPath(BundleType.G));
            AssetBundleGeneratorV2.miniMode = true;
            var bundleBuilds = ProcessFolderBundleTag2.CollectBundle(target, buildParams);
            
            //use File Name addressableNames
            _allAddressNames.Clear();
            for (var i = 0; i < bundleBuilds.Length; i++)
            {
	            bundleBuilds[i].addressableNames = GetAddressableNamesList(bundleBuilds[i].assetNames).ToArray();
            }
            var buildContent = new BundleBuildContent(bundleBuilds);
            // Populate the bundle specific compression data
            //static和缓存的bundle用lzma压缩
            foreach (var build in bundleBuilds)
            {
	            if (AssetConfig.BundleNeedDecompress(build.assetBundleName))
	            {
		            buildParams.PerBundleCompression.Add(build.assetBundleName,BuildCompression.Uncompressed);
	            }
            }
        }
        public static void DeletePath(string fullPath)
        {
            string locationPathName = Path.GetFullPath(".") + "/../../build/";
            if (Directory.Exists(fullPath))
            {
	            Directory.Delete(fullPath, true);
	            return;
#if UNITY_EDITOR_WIN
                using var process = new System.Diagnostics.Process
                {
                    StartInfo = new System.Diagnostics.ProcessStartInfo
                    {
                        RedirectStandardOutput = true,
                        UseShellExecute = false,
                        CreateNoWindow = true,
                        WindowStyle = System.Diagnostics.ProcessWindowStyle.Hidden,
                        FileName = Path.Combine(locationPathName, "../buildtool/delete.bat"),
                        Arguments = fullPath
                    }

                };
                process.ErrorDataReceived += Process_ErrorDataReceived;
                process.Start();
                process.WaitForExit();             
#else
	            Directory.Delete(fullPath, true);
#endif
            }
        }

        private static void Process_ErrorDataReceived(object sender, System.Diagnostics.DataReceivedEventArgs e)
        {
			Debug.LogError("delete.bat error happened");
			Debug.LogError(e.ToString());
        }

        public new static void GenerateAssetBundleForAndroid(string resVersion = "0.0.0.0")
		{
			
#if UNITY_2020_2_OR_NEWER && !USE_SPRITE_ATLAS
			EditorSettings.spritePackerMode = SpritePackerMode.AlwaysOn;
			UnityEditor.Sprites.Packer.RebuildAtlasCacheIfNeeded(BuildTarget.Android, false);
#endif
			string bundlePath = Application.dataPath + "/StreamingAssets/GameAssets/AssetBundle2";
			DeletePath(bundlePath);
			
			AssetBundleOutputPath = ANDROID_BUNDLE_PATH;
			
			CompatibilityAssetBundleManifest m = BuildBundle(BuildTarget.Android);
			
			AppendExtension();

			MoveOtaToR();

			GenerateAssetBundleImplement(BundleType.R, m);

			MoveWwiseBankToG("Android");
			
			MoveCustomBundleToG("Android");
			
			GenerateAssetBundleImplement(BundleType.G, m);

			//强制使用旧版本bundle环节
			ProcessCertainPrefabs(true);

            AssetDatabase.Refresh();
            OnBundleBuilt();
        }

"""

In [7]:
prefix = """# test.py
import random
def test_print():
    # 以下代码将打印出
"""
suffix = """
    ret = random.randint(1000,9999)
    print(ret)"""

In [8]:
prefix = """// main.go
resp, err := c.llm.CreateCompletions(ctx, completionReq)"""
suffix = """return "", err
}"""

In [9]:
from langchain_core.output_parsers.base import BaseOutputParser
class CustOuputParser(BaseOutputParser[str]):  
    
    def parse(self, text: str) -> str:
        cleaned_text = text.replace("<fim_middle>","")
        return cleaned_text

    @property
    def _type(self) -> str:
        return "cust_output_parser"

In [10]:
output_parser = CustOuputParser()
chain_haiku = prompt_template | llm_haiku.bind(stop=["</fim_middle>"]) |output_parser
chain_sonnet = prompt_template | llm_sonnet.bind(stop=["</fim_middle>"]) |output_parser
chain_gpt = prompt_template | llm_gpt.bind(stop=["</fim_middle>"]) |output_parser
chain_gpt4 = prompt_template | llm_gpt4.bind(stop=["</fim_middle>"]) |output_parser

In [11]:
resp_haiku= chain_haiku.invoke({"code_lang":"go","prefix":prefix,"suffix":suffix})
print(resp_haiku)

if err != nil {
    return "", err
}
return resp.Choices[0].Text, nil



In [226]:
print(prefix+'\n'+resp_haiku+'\n'+suffix)

// main.go
resp, err := c.llm.CreateCompletions(ctx, completionReq)

if err != nil {
    return "", err
}
if resp.Choices == nil || len(resp.Choices) == 0 {
    return "", fmt.Errorf("no completions generated")
}
return resp.Choices[0].Text, nil

return "", err
}


In [227]:
resp_sonnet= chain_sonnet.invoke({"code_lang":"go","prefix":prefix,"suffix":suffix})
print(resp_sonnet)


if err != nil {
    return "", err
}



In [228]:
print(prefix+'\n'+resp_sonnet+'\n'+suffix)

// main.go
resp, err := c.llm.CreateCompletions(ctx, completionReq)

if err != nil {
    return "", err
}

return "", err
}


In [229]:
resp_gpt35= chain_gpt.invoke({"code_lang":"go","prefix":prefix,"suffix":suffix})
print(resp_gpt35)

resp, err := c.llm.CreateCompletions(ctx, completionReq)
if err != nil {
    return "", err
}
return resp, nil



In [230]:
print(prefix+'\n'+resp_gpt35+'\n'+suffix)

// main.go
resp, err := c.llm.CreateCompletions(ctx, completionReq)
resp, err := c.llm.CreateCompletions(ctx, completionReq)
if err != nil {
    return "", err
}
return resp, nil

return "", err
}


In [231]:
resp_gpt4= chain_gpt4.invoke({"code_lang":"go","prefix":prefix,"suffix":suffix})
print(resp_gpt4)

if err != nil {
    return "", err
}
completion := string(resp.Data)
return completion, nil



In [232]:
print(prefix+'\n'+resp_gpt4+'\n'+suffix)

// main.go
resp, err := c.llm.CreateCompletions(ctx, completionReq)
if err != nil {
    return "", err
}
completion := string(resp.Data)
return completion, nil

return "", err
}
